<a href="https://colab.research.google.com/github/sambhavpurohit14/Smart_Gallery/blob/encoder-functions/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
'''
sentence -> tokens -> inputs ID (location in the vocab) -> embedding
'''
import torch
import torch.nn as nn
import math

In [8]:
'''
    embedding maps the numbers to a vector, and those numbers are learned during training
     in paper multiply the embeddings by sqrt(model_dimensions)
'''
class SentenceEmbeddings(nn.Module):
    def __init__(self, model_dimensions: int, vocab_size: int):
        super().__init__()
        self.model_dimensions = model_dimensions
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(self.vocab_size, self.model_dimensions)

    def forward(self, inputs):
        return self.embedding(inputs) * math.sqrt(self.model_dimensions)


In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, model_dimensions: int, max_length: int, dropout: float):
        super().__init__()
        self.model_dimensions = model_dimensions
        self.max_length = max_length
        self.dropout = nn.Dropout(dropout)

        position_encoding = torch.zeros(max_length, model_dimensions)
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)
        denominator = torch.exp(torch.arange(0, model_dimensions, 2).float() * (-math.log(10000.0) / model_dimensions))

        position_encoding[:, 0::2] = torch.sin(position * denominator)
        position_encoding[:, 1::2] = torch.cos(position * denominator)

        position_encoding = position_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('positional_encoding', position_encoding)

    def forward(self, inputs):
        inputs = inputs + self.positional_encoding[:inputs.size(0), :]
        return self.dropout(inputs)


In [10]:
'''
layer normalization - calculate mean and variance for each item independently of others

'''
class LayerNormalization(nn.Module):
    def __init__(self, epsilon: float = 1e-8):
        super().__init__()
        self.epsilon = epsilon
        self.w = nn.Parameter(torch.ones(1))
        self.b = nn.Parameter(torch.zeros(1))

    def forward(self, inputs):
        mean = inputs.mean(dim=-1, keepdim=True)
        std = inputs.std(dim=-1, keepdim=True)
        return self.w * (inputs - mean) / (std + self.epsilon) + self.b

In [11]:
class FeedForward(nn.Module):
    def __init__(self, model_dimensions: int, ff: int, dropout: float):
        super().__init__()
        self.linear_1 = nn.Linear(model_dimensions, ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(ff, model_dimensions)

    def forward(self, inputs):
        return self.linear_2(self.dropout(torch.relu(self.linear_1(inputs))))

In [12]:
''' multihead attention
define 3 matrices q, k, v
in case of encoder, all are same
W is learned

attention = softmax((QK^T) / sqrt(d_k)
get 1D heads, multiply by Wo
 '''
class MultiHeadAttention(nn.Module):
    def __init__(self, model_dimensions: int, h: int, dropout: float):
        super().__init__()
        self.model_dimensions = model_dimensions
        self.h = h
        assert model_dimensions % h == 0, "Model dimensions must be divisible by the number of heads."
        self.d_k = model_dimensions // h

        self.w_q = nn.Linear(model_dimensions, model_dimensions)
        self.w_k = nn.Linear(model_dimensions, model_dimensions)
        self.w_v = nn.Linear(model_dimensions, model_dimensions)
        self.w_o = nn.Linear(model_dimensions, model_dimensions)
        self.dropout = nn.Dropout(dropout)

    def attention(self, query, key, value, mask=None):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention_weights = torch.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        return torch.matmul(attention_weights, value), attention_weights

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)

        query = self.w_q(q).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
        key = self.w_k(k).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
        value = self.w_v(v).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)

        x, _ = self.attention(query, key, value, mask)

        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.model_dimensions)
        return self.w_o(x)

In [13]:
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float):
        super().__init__()
        self.layer_norm = LayerNormalization()
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, sublayer):
        return inputs + self.dropout(sublayer(self.layer_norm(inputs)))


In [14]:
class EncoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttention, feed_forward_block: FeedForward, dropout: float):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self, inputs, mask):
        x = self.residual_connections[0](inputs, lambda x: self.self_attention_block(x, x, x, mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

In [15]:
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, inputs, mask):
        x = inputs
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [16]:
sentence = "The quick brown fox jumps over the lazy dog"